In [1]:
import whisper
from pprint import pprint

In [3]:
# Load the model (you can use "base", "small", "medium", or "large" for better accuracy)
model = whisper.load_model("medium")  

In [5]:
# file_path = r"C:\Users\User_1\Desktop\Comm-sounds\sat.mp3"
file_path = r"C:\Users\User_1\Desktop\Comm-sounds\1.mp3"
# file_path = r"C:\Users\User_1\Desktop\Comm-sounds\2.mp3"
# file_path = r"C:\Users\User_1\Desktop\Comm-sounds\3.mp3"
# file_path = r"C:\Users\User_1\Desktop\Comm-sounds\fa_1.m4a"

In [24]:
# Transcribe Persian audio
LANGUAGE = "en"
# LANGUAGE = "fa"

result = model.transcribe(file_path, language=LANGUAGE)

# Print Persian transcription
pprint(result["text"])

(' 300 checks. 300. Dangerous check stabilizers. Sit at 75%. Copy. Standing '
 'by. DOB LPC, turn fine. Declining memory. Turn back for an unexpected A. Up '
 'speed. DOB LPC, up speed successful with no unexpected errors. Copy, DOB. '
 'You are clear for launch. And with that, shut down your visors. O2 on and '
 "prepare for ignition, O2. Copy that and initiating a full 10-4-4. You've got "
 'to check two of your own. LT, L1 clear. Engines on. CP13, sector on. Wing '
 'turning stable. O L H. Woo! Just getting the rock and roll. Over tone stage. '
 'Alright, LL 60. Two minutes 45. Calling two engines manual. Copy, two '
 "engines manual. Turning, we'll be back at 8.2, check. Standing by the full "
 'trails. Overlight deck chairs. On tour manual. 2.4, 2.8G right now and '
 'counting. Access distortion negative one. Standby.')


In [5]:
import torchaudio
import librosa
import torch
from df import enhance, init_df
import soundfile as sf

# Load DeepFilterNet model
model, df_state, _ = init_df()  # Load default model

# File path to your noisy MP3 file
file_path = r"C:\Users\User_1\Desktop\Comm-sounds\2.mp3"
output_file_path = r"C:\Users\User_1\Desktop\Comm-sounds\output_1.wav"

# Load the audio file as a NumPy array
noisy_audio, sr = librosa.load(file_path, sr=16000)  # Resample to 16kHz if needed

# Convert to PyTorch tensor
noisy_audio_tensor = torch.tensor(noisy_audio).unsqueeze(0)  # Add batch dimension

# Enhance the audio
enhanced_audio = enhance(model, df_state, noisy_audio_tensor)

# Convert output to NumPy and save as WAV
sf.write(output_file_path, enhanced_audio.squeeze().cpu().numpy(), sr)

print("✅ Enhanced audio saved")


2025-03-06 09:04:58 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-03-06 09:04:58 | INFO     | DF | Using DeepFilterNet3 model at C:\Users\User_1\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3
2025-03-06 09:04:58 | INFO     | DF | Initializing model `deepfilternet3`
2025-03-06 09:04:58 | INFO     | DF | Found checkpoint C:\Users\User_1\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3\checkpoints\model_120.ckpt.best with epoch 120
2025-03-06 09:04:58 | INFO     | DF | Running on device cuda:0
2025-03-06 09:04:58 | INFO     | DF | Model loaded
✅ Enhanced audio saved


In [17]:
import torchaudio
import librosa
import torch
from df import enhance, init_df
import soundfile as sf
import torchaudio.functional as F

# Load DeepFilterNet model
model, df_state, _ = init_df()  # Ensure the model is correctly loaded

# File path to your noisy MP3 file
file_path = r"C:\Users\User_1\Desktop\Comm-sounds\sat.mp3"
output_file_path = r"C:\Users\User_1\Desktop\Comm-sounds\output_1.wav"


# Load the audio file using librosa
noisy_audio, sr = librosa.load(file_path, sr=16000)  # Resample to 16kHz
noisy_audio = librosa.util.normalize(noisy_audio)  # Normalize amplitude
noisy_audio, _ = librosa.effects.trim(noisy_audio)  # Trim silence

# Convert to PyTorch tensor and ensure the correct shape
noisy_audio_tensor = torch.tensor(noisy_audio, dtype=torch.float32).unsqueeze(0)  # Shape: (1, samples)

# Ensure model is in evaluation mode
model.eval()

# Apply DeepFilterNet Enhancement with stronger noise suppression
enhanced_audio = enhance(model, df_state, noisy_audio_tensor, atten_lim_db=-40)  # Stronger filtering

# Apply a high-pass filter to remove low-frequency noise
enhanced_audio = torchaudio.functional.highpass_biquad(enhanced_audio, sr, cutoff_freq=200)  # Removes rumbling noise

# Apply post-processing: Boost speech frequencies if needed
enhanced_audio = F.gain(enhanced_audio, gain_db=12)  # Boost speech clarity

# Save the enhanced audio as WAV
sf.write(output_file_path, enhanced_audio.squeeze().cpu().numpy(), sr)

print("✅ Strongly Enhanced Audio Saved as 'enhanced_output.wav'")


2025-03-06 09:17:14 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-03-06 09:17:14 | INFO     | DF | Using DeepFilterNet3 model at C:\Users\User_1\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3
2025-03-06 09:17:14 | INFO     | DF | Initializing model `deepfilternet3`
2025-03-06 09:17:14 | INFO     | DF | Found checkpoint C:\Users\User_1\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3\checkpoints\model_120.ckpt.best with epoch 120
2025-03-06 09:17:14 | INFO     | DF | Running on device cuda:0
2025-03-06 09:17:14 | INFO     | DF | Model loaded
✅ Strongly Enhanced Audio Saved as 'enhanced_output.wav'
